In [1]:
import simuvex
import angr

In [2]:
# Hacklu 2013 - reverse_me
# @author: P1kachu <p1kachu@lse.epita.fr>

In [3]:
path_types = [ 
    #'avoid',
    'errored',
    'deadended',
    'found',
    'pruned',
    'stashed',
    'unconstrained',
    'unsat'
]
def print_paths(ex, trace=False):
    for p_type in path_types:
        for path in getattr(ex, p_type):
            print("")
            print("{0}: {1}".format(p_type, path))
            if p_type == 'errored':
                print("Error: {0}".format(path.error))
            if trace:
                for step in path.trace:
                    print(step)

def i_am_clean(state):
    # Bypass ptrace/ld_peload checks
    print("I am clean at {0}".format(state.regs.eip))
    state.regs.eax = 0

def unconstrained(state):
    print("Hooked at {0}".format(state.regs.eip))

In [4]:
BINARY = 'reverse_me.bin'
fail = (0x08048e18, 0x08048711)
win  = (0x08048e16)
main = 0x080486f7
flag_addr = 0xd0000010
argv_addr = 0xd0000000

# Need something symbolic, we don't have the length !
PASS_LEN = 100

In [5]:
p = angr.Project(BINARY)
    
p.hook(main + 0xf, func=i_am_clean, length=5)

In [6]:
init = p.factory.blank_state(addr=main)
argv=[BINARY, init.se.BVS('arg1', PASS_LEN * 8)]

init.memory.store(argv_addr, argv[0])
init.memory.store(flag_addr, argv[1])
init.regs.edi = argv_addr 
init.regs.esi = 2 # argc

In [7]:
pgp = p.factory.path_group(init)

In [8]:
ex = pgp.explore(find=win, avoid=fail)
print(ex)

I am clean at <SAO <BV32 0x8048706>>
<PathGroup with 5 avoid, 1 found, 6 errored>


In [9]:
#print_paths(ex, trace=True)
print(ex.found[0])

<Path with 1806 runs (at 0x8048e16)>


In [10]:
s = ex.found[0].state
tmp = s.se.any_str(s.memory.load(flag_addr, 40 * 8))
print(tmp)
x = s.simplify()
for y in x[0].args:
    print(y)

                                                                                                                                                                                                                                                                                                                                
<Bool mem_7fff0004_3_32 == 0x2>
<Bool mem_7fff0008_5_32 == 0xfffffffc>
<Bool mem_0_6_32 == 0x0>
<Bool mem_c0000000_18_8 == 17>
<Bool mem_c0000004_22_8 == 97>
<Bool mem_c0000008_26_8 == 102>
<Bool mem_c000000c_30_8 == 75>
<Bool mem_c0000001_19_8 == 96>
<Bool mem_c0000002_20_8 == 50>
<Bool mem_c0000003_21_8 == 88>
<Bool mem_c0000005_23_8 == 101>
<Bool mem_c0000006_24_8 == 81>
<Bool mem_c0000007_25_8 == 34>
<Bool mem_c0000009_27_8 == 98>
<Bool mem_c000000a_28_8 == 107>
<Bool mem_c000000b_29_8 == 94>
<Bool mem_c000000d_31_8 == 69>
<Bool mem_c000000e_32_8 == 110>
<Bool mem_c000000f_33_8 == 85>
